# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# !pip install python-dotenv
import os
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
geoapify_key = os.getenv("geoapify_key")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City-ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,3.29,75,40,6.71,CA,1712523518
1,1,margaret river,-33.9500,115.0667,13.25,76,1,6.82,AU,1712523527
2,2,papatowai,-46.5619,169.4708,10.11,89,97,1.57,NZ,1712523322
3,3,kathmandu,27.7167,85.3167,16.12,48,20,1.54,NP,1712523514
4,4,keflavik,64.0049,-22.5624,0.95,74,75,11.32,IS,1712523184


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
)

# Display the map plot
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
countries = ["ES", "JP", "SE"]
filtered_cities = city_data_df.loc[city_data_df["Country"].isin(countries)]
#& (city_data_df["Lat"]<31) \
#                & (city_data_df["Lat"]>=20) &(city_data_df["Lng"]<115) & (city_data_df["Lat"]>=-150)

# Drop any rows with null values
city_data_df.dropna(how="any")

# # Display sample data
filtered_cities.head()

,City-ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,hirara,24.8000,125.2833,23.92,100,75,3.60,JP,1712523566
62,62,arno,60.1167,18.6333,7.66,76,5,7.94,SE,1712523588
134,134,nemuro,43.3236,145.5750,3.57,100,100,4.02,JP,1712523241
292,292,nanao,37.0500,136.9667,10.15,80,100,1.54,JP,1712523729
311,311,naze,28.3667,129.4833,19.51,92,100,3.77,JP,1712523264


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

/var/folders/_7/hzy3hwcs7q72sm8v1h1k2rq80000gn/T/ipykernel_6400/526122274.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
17,hirara,JP,24.8000,125.2833,100,
62,arno,SE,60.1167,18.6333,76,
134,nemuro,JP,43.3236,145.5750,100,
292,nanao,JP,37.0500,136.9667,80,
311,naze,JP,28.3667,129.4833,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary and convert to API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hirara - nearest hotel: 古謝そば屋
arno - nearest hotel: Pensionat Grisslehamns
nemuro - nearest hotel: お宿エクハシ
nanao - nearest hotel: ホテル・アルファ－ワン能登和倉
naze - nearest hotel: ビジネス旅館畠山
katsuren-haebaru - nearest hotel: No hotel found
zaragoza - nearest hotel: Hotel Pilar Plaza
hasaki - nearest hotel: Choshi Plaza Hotel
iwanai - nearest hotel: ホチルいのう
carboneras - nearest hotel: CaSabah
agramunt - nearest hotel: Hotel Restaurant Blanc & Negre 2
valence - nearest hotel: NALU Boutique Rooms


,City,Country,Lat,Lng,Humidity,Hotel Name
17,hirara,JP,24.8000,125.2833,100,古謝そば屋
62,arno,SE,60.1167,18.6333,76,Pensionat Grisslehamns
134,nemuro,JP,43.3236,145.5750,100,お宿エクハシ
292,nanao,JP,37.0500,136.9667,80,ホテル・アルファ－ワン能登和倉
311,naze,JP,28.3667,129.4833,92,ビジネス旅館畠山
361,katsuren-haebaru,JP,26.3369,127.8719,96,No hotel found
424,zaragoza,ES,41.6561,-0.8773,63,Hotel Pilar Plaza
436,hasaki,JP,35.7333,140.8333,90,Choshi Plaza Hotel
467,iwanai,JP,42.9744,140.5089,75,ホチルいのう
491,carboneras,ES,37.0015,-1.9213,68,CaSabah


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "Humidity",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)